<a href="https://colab.research.google.com/github/MLFlexer/nlp-course/blob/malthe/bert_seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Made with the help of this guide: https://huggingface.co/docs/transformers/tasks/question_answering


In [1]:
!pip install datasets transformers evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("copenlu/answerable_tydiqa")

train_set = dataset["train"]
validation_set = dataset["validation"]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/116067 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13325 [00:00<?, ? examples/s]

In [22]:
dataset["train"][0]

{'question_text': 'Milloin Charles Fort syntyi?',
 'document_title': 'Charles Fort',
 'language': 'finnish',
 'annotations': {'answer_start': [18],
  'answer_text': ['6. elokuuta (joidenkin lähteiden mukaan 9.) 1874']},
 'document_plaintext': 'Charles Hoy Fort (6. elokuuta (joidenkin lähteiden mukaan 9.) 1874 – 3. toukokuuta 1932) oli yhdysvaltalainen kirjailija ja paranormaalien ilmiöiden tutkija.',
 'document_url': 'https://fi.wikipedia.org/wiki/Charles%20Fort'}

In [28]:
filtered_dataset = dataset.filter(lambda entry: entry["language"] in ["english"])

In [29]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

In [21]:
filtered_dataset["train"][0]
annotations = filtered_dataset["train"]["annotations"]
i=0
start = annotations[i]["answer_start"][0]
answer = annotations[i]["answer_text"][0]
end = annotations[i]["answer_start"][0] + len(answer)

preprocess_function(filtered_dataset["train"][0])

TypeError: ignored

In [19]:
def preprocess_function(dataset):
    questions = [q.strip() for q in dataset["question_text"]]
    inputs = tokenizer(
        questions,
        dataset["document_plaintext"],
        max_length=384, # TODO: skift dette
        truncation="only_second", # TODO: skift dette
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    print(offset_mapping)
    annotations = dataset["annotations"]
    answers = []
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        start = annotations[i]["answer_start"][0]
        if start == -1:
          start_positions.append(0)
          end_positions.append(0)
        else:
          answer = annotations[i]["answer_text"][0]
          end = annotations[i]["answer_start"][0] + len(answer)
          start_positions.append(answer)
          end_positions.append(end)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


"""
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
inputs["start_positions"] = start_positions
inputs["end_positions"] = end_positions
return inputs"""

'\n        sequence_ids = inputs.sequence_ids(i)\n\n        # Find the start and end of the context\n        idx = 0\n        while sequence_ids[idx] != 1:\n            idx += 1\n        context_start = idx\n        while sequence_ids[idx] == 1:\n            idx += 1\n        context_end = idx - 1\n\n        # If the answer is not fully inside the context, label it (0, 0)\n        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:\n            start_positions.append(0)\n            end_positions.append(0)\n        else:\n            # Otherwise it\'s the start and end token positions\n            idx = context_start\n            while idx <= context_end and offset[idx][0] <= start_char:\n                idx += 1\n            start_positions.append(idx - 1)\n\n            idx = context_end\n            while idx >= context_start and offset[idx][1] >= end_char:\n                idx -= 1\n            end_positions.append(idx + 1)\ninputs["start_positions"] = st

In [23]:
dataset["train"][0]

{'question_text': 'Milloin Charles Fort syntyi?',
 'document_title': 'Charles Fort',
 'language': 'finnish',
 'annotations': {'answer_start': [18],
  'answer_text': ['6. elokuuta (joidenkin lähteiden mukaan 9.) 1874']},
 'document_plaintext': 'Charles Hoy Fort (6. elokuuta (joidenkin lähteiden mukaan 9.) 1874 – 3. toukokuuta 1932) oli yhdysvaltalainen kirjailija ja paranormaalien ilmiöiden tutkija.',
 'document_url': 'https://fi.wikipedia.org/wiki/Charles%20Fort'}

In [32]:
def preprocess_function(entry):
    questions = [q.strip() for q in entry["question_text"]]
    inputs = tokenizer(
        questions,
        entry["document_plaintext"],
        max_length=384, # TODO: skift dette
        truncation="only_second", # TODO: skift dette
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    annotations = entry["annotations"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = annotations[i]["answer_text"]
        start_char = annotations["answer_start"][0]
        end_char = annotations["answer_start"][0] + len(answer[0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [34]:
preprocess_function()

KeyError: ignored

In [43]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question_text"]]
    contexts = examples["document_plaintext"]
    #answers = examples["annotations"]["answer_text"]
    #answer_start = examples["annotations"]["answer_start"]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        # Find the start and end offsets for the answer
        answer = examples["annotations"][i]["answer_text"][0]  # Assuming we use the first answer if there are multiple
        start_char = examples["annotations"][i]["answer_start"][0]
        end_char = start_char + len(answer)

        for idx, (start, end) in enumerate(offsets):
            if start <= start_char <= end:
                start_positions.append(idx)
            if start <= end_char <= end:
                end_positions.append(idx)

    # If there are no start positions, set them to 0
    if not start_positions:
        start_positions.append(0)
    # If there are no end positions, set them to the max position
    if not end_positions:
        end_positions.append(len(offset_mapping) - 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [44]:
tokenized_data = filtered_dataset.map(preprocess_function, batched=True, remove_columns=filtered_dataset["train"].column_names)

Map:   0%|          | 0/7389 [00:00<?, ? examples/s]

ArrowInvalid: ignored